# Transforming files to parameters.
## Should run after "Contact Patterns" notebook!!

### In order to run this notebook One should make sure the above files exist:
* Data/raw
    * Data/raw/asymptomatic proportions.csv
    * Data/raw/population size.csv                    ============MISSING============
    * Data/raw/init_i_italy.csv                       ============MISSING============
    * Data/raw/positive_prop_region_2703.csv          ============MISSING============
    * Data/raw/IL_initial_eps.csv                     ============MISSING============
    * Data/raw/hospitalizations.csv                   ============MISSING============
    
* Data/base_contact_mtx
    * Data/base_contact_mtx/full_home_mtx.csv
    * Data/base_contact_mtx/full_leisure_mtx_no_school.csv
    * Data/base_contact_mtx/full_leisure_mtx_no_work.csv
    * Data/base_contact_mtx/full_leisure_mtx_routine.csv
    * Data/base_contact_mtx/full_work_mtx_no_school.csv
    * Data/base_contact_mtx/full_work_mtx_no_work.csv
    * Data/base_contact_mtx/full_work_mtx_routine.csv
    
* Data/demograph
    * Data/demograph/init_pop.pickle                  ???????????????????????????????
    * Data/demograph/religion_dis.csv

### This Notebook generates the above files:
* Data/parameters
    - Data/parameters/f0_full.pickle
    - Data/parameters/eps_dict.pickle
    - Data/parameters/eps_by_region.pickle
    - Data/parameters/hospitalization.pickle
    - Data/parameters/C_calibration.pickle
    - Data/parameters/orthodox_dist.pickle

# Imports

In [10]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
from scipy import optimize
import sys
import os
sys.path.append('../SEIR_full/')
sys.path.append('..')
from SEIR_full import *
%matplotlib inline

In [2]:
N

{0: ('Intervention', '11', 'High', '0-4'),
 1: ('Intervention', '11', 'High', '5-9'),
 2: ('Intervention', '11', 'High', '10-19'),
 3: ('Intervention', '11', 'High', '20-29'),
 4: ('Intervention', '11', 'High', '30-39'),
 5: ('Intervention', '11', 'High', '40-49'),
 6: ('Intervention', '11', 'High', '50-59'),
 7: ('Intervention', '11', 'High', '60-69'),
 8: ('Intervention', '11', 'High', '70+'),
 9: ('Intervention', '11', 'Low', '0-4'),
 10: ('Intervention', '11', 'Low', '5-9'),
 11: ('Intervention', '11', 'Low', '10-19'),
 12: ('Intervention', '11', 'Low', '20-29'),
 13: ('Intervention', '11', 'Low', '30-39'),
 14: ('Intervention', '11', 'Low', '40-49'),
 15: ('Intervention', '11', 'Low', '50-59'),
 16: ('Intervention', '11', 'Low', '60-69'),
 17: ('Intervention', '11', 'Low', '70+'),
 18: ('Intervention', '11_betshemesh', 'High', '0-4'),
 19: ('Intervention', '11_betshemesh', 'High', '5-9'),
 20: ('Intervention', '11_betshemesh', 'High', '10-19'),
 21: ('Intervention', '11_betshemesh

# Asymptomatic

In [3]:
asymp = pd.read_csv('../Data/raw/asymptomatic proportions.csv', index_col=0)
asymp.head()

,Scenario 1,Scenario 2,Scenario 3,Unnamed: 4
Age,,,,
0-4,0.957467,0.765973,0.612779,NaN
5-9,0.957467,0.765973,0.612779,NaN
10-19,0.972167,0.777734,0.622187,NaN
20-29,0.921424,0.737139,0.589711,NaN
30-39,0.899804,0.719843,0.575874,NaN


In [9]:
f0_full = {} # dict that contains the possible scenarios

# asymptomatic with risk group, high risk with 0
f_init = np.zeros(len(list(itertools.product(R.values(), A.values()))))
for i in [1,2,3]:
    f_tmp = f_init.copy()
    f_tmp[9:] = asymp['Scenario '+ str(i)].values[:-1]
    f0_full['Scenario '+ str(i)] = expand_partial_array(risk_age_dict, f_tmp)

In [11]:
# Save
try:
    os.mkdir('../Data/parameters')
except:
    pass
with open('../Data/parameters/f0_full.pickle', 'wb') as handle:
    pickle.dump(f0_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Initial illness

In [37]:
# Age dist. positive specimens
age_dist = {'0-4':0.02, '5-9':0.02, '10-19':0.11, '20-29':0.23, '30-39':0.15, '40-49':0.14, '50-59':0.14, '60-69':0.11,
           '70+':0.08}

In [38]:
# Age distribution:
with open('../Data/demograph/init_pop.pickle', 'rb') as f:
    pop_dist = pickle.load(f)
    
# risk distribution by age:
risk_dist = pd.read_csv('../Data/population size.csv')

init_I_dis = pd.read_csv('../Data/init_i_italy.csv')['proportion'].values[:-1]

eps_t = {}

FileNotFoundError: [Errno 2] No such file or directory: '../Data/demograph/init_pop.pickle'

In [5]:
init_I_dis.sum()

0.9999999999999999

In [151]:
# Scenario 1
init_I_IL1 = (491./(1-(f_init1 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis1 = init_I_dis * init_I_IL1

# Scenario 2
init_I_IL2 = (491./(1-(f_init2 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis2 = init_I_dis * init_I_IL2

# Scenario 3
init_I_IL3 = (491./(1-(f_init3 * risk_dist['pop size'].values).sum())) / 9136000.
init_I_dis3 = init_I_dis * init_I_IL3

In [64]:
eps_sc1 = []
for i in range(1000):
    eps_sc1.append(init_I_dis1[i] * pop_dist if i < len(init_I_dis1) else 0)

eps_sc2 = []
for i in range(1000):
    eps_sc2.append(init_I_dis2[i] * pop_dist if i < len(init_I_dis2) else 0)

eps_sc3 = []
for i in range(1000):
    eps_sc3.append(init_I_dis3[i] * pop_dist if i < len(init_I_dis3) else 0)
    
    
eps_t['Scenario1'] = eps_sc1
eps_t['Scenario2'] = eps_sc2
eps_t['Scenario3'] = eps_sc3

In [65]:
# Save
with open('../Data/parameters/eps_dict.pickle', 'wb') as handle:
    pickle.dump(eps_t, handle, protocol=pickle.HIGHEST_PROTOCOL)

## eps by region proportion

In [133]:
# Loading data
region_prop = pd.read_csv('../Data/raw/positive_prop_region_2703.csv', index_col=0)['cases_prop'].copy()
risk_prop = pd.read_csv('../Data/raw/risk_dist.csv', index_col=0)['risk'].copy()
print(region_prop.head())
print()
print(risk_prop.head())

sub_district_id
11               0.137496
11_betshemesh    0.012624
21               0.010918
22               0.006482
23               0.037189
Name: cases_prop, dtype: float64

Age
0-4      0.050
5-9      0.106
10-19    0.106
20-29    0.149
30-39    0.149
Name: risk, dtype: float64


In [97]:
eps_t_region = {}

In [152]:
for sc,init_I in [('Scenario1',init_I_dis1),('Scenario2',init_I_dis2),('Scenario3',init_I_dis3)]:
    eps_temp = []
    for t in range(1000):
        if t < len(init_I):
            # empty array for day t
            day_vec = np.zeros(len(mdl.N))
            # fill in the array, zero for intervention groups
            for key in mdl.N.keys():
                if mdl.N[key][0] == 'Intervention':
                    day_vec[key] = 0
                else:
                    day_vec[key] = init_I[t] * region_prop[mdl.N[key][1]] * age_dist[mdl.N[key][3]] * \
                    (risk_prop[mdl.N[key][3]]**(1 - (mdl.N[key][2] == 'Low'))) * \
                    ((1 - risk_prop[mdl.N[key][3]])**(mdl.N[key][2] == 'Low'))
            eps_temp.append(day_vec)
        else:
             eps_temp.append(0.0)
        
        eps_t_region[sc] = eps_temp


In [168]:
# save eps:
with open('../Data/parameters/eps_by_region.pickle', 'wb') as handle:
    pickle.dump(eps_t_region, handle, protocol=pickle.HIGHEST_PROTOCOL)  

## Smoothed eps by region

In [13]:
init_eps = pd.read_csv('../Data/raw/IL_initial_eps.csv', header=None).values[:,0].copy()
init_eps

array([  0.78180071,   2.94105921,   5.84488904,   8.5253491 ,
        12.02483762,  17.497441  ,  24.38473153,  33.09622265,
        41.99385742,  39.83459892,  75.79742365,  93.14595037,
       114.9991361 , 143.6279275 , 172.5917768 , 219.0158361 ,
       274.5609032 , 341.4606921 ])

In [51]:
# Scenario 1
init_eps_asymp1 = (init_eps/(1-(f_init1 * risk_dist['pop size'].values).sum())) / 9136000.

# Scenario 2
init_eps_asymp2 = (init_eps/(1-(f_init2 * risk_dist['pop size'].values).sum())) / 9136000.


# Scenario 3
init_eps_asymp3 = (init_eps/(1-(f_init3 * risk_dist['pop size'].values).sum())) / 9136000.


In [26]:
f_init3

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.6127786 ,
       0.6127786 , 0.62218712, 0.58971125, 0.57587446, 0.59021178,
       0.56415614, 0.54046208, 0.14933333])

In [53]:
init_eps_asymp3.sum()*9136000

3003.15972721764

In [54]:
def fill_eps_day(init_eps, dic=mdl.N, age_)

{0: '11',
 1: '11_betshemesh',
 2: '21',
 3: '22',
 4: '23',
 5: '24',
 6: '29',
 7: '31',
 8: '32',
 9: '41',
 10: '42',
 11: '43',
 12: '44',
 13: '51',
 14: '51_tlv',
 15: '51_bb',
 16: '61',
 17: '62',
 18: '62_arab',
 19: '71'}

In [ ]:
eps_reg = []


# Hospitalizations

In [35]:
hosp_init = pd.read_csv('../Data/raw/hospitalizations.csv')
hosp_init

,Age,risk,pr_hospital
0,0-4,High,0.029022
1,5-9,High,0.025129
2,10-19,High,0.025129
3,20-29,High,0.045566
4,30-39,High,0.113915
5,40-49,High,0.136698
6,50-59,High,0.163551
7,60-69,High,0.207688
8,70+,High,0.184739
9,0-4,Low,0.006894


In [36]:
hosp = expand_partial_array(risk_age, hosp_init['pr_hospital'].values)

In [38]:
# Save
with open('../Data/parameters/hospitalization.pickle', 'wb') as handle:
    pickle.dump(hosp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Calibration contact matrix

In [28]:
C_calibration['work_non'][-1]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.46220555e+00, 6.26241406e+00, ...,
        3.81902449e-04, 2.86615769e-05, 2.91338586e-08],
       [0.00000000e+00, 4.84603341e+00, 2.54064586e+01, ...,
        2.02862940e-03, 2.04167704e-04, 1.11372472e-07],
       ...,
       [0.00000000e+00, 1.06193849e-04, 3.60858148e-04, ...,
        8.87325030e+01, 5.93465184e+00, 3.39557424e-03],
       [2.23698024e-09, 8.86264533e-06, 4.45729034e-05, ...,
        1.05649857e+01, 8.23320798e-01, 5.67847443e-03],
       [0.00000000e+00, 3.73510417e-09, 3.56288951e-08, ...,
        2.53169173e-03, 2.53738863e-03, 2.72413300e-03]])

Important : here no intervention mean the course of interventions made by government

In [29]:
full_mtx_home = pd.read_csv('../Data/base_contact_mtx/full_home_mtx.csv', index_col=[0,1],header=[0, 1])
full_mtx_work = {'routine':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_routine.csv', index_col=[0,1],header=[0, 1]),
                        'no_school':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_no_school.csv', index_col=[0,1],header=[0, 1]),
                        'no_work':pd.read_csv('../Data/base_contact_mtx/full_work_mtx_no_work.csv', index_col=[0,1],header=[0, 1])}

full_mtx_leisure = {'routine':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_routine.csv', index_col=[0,1],header=[0, 1]),
                        'no_school':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_no_school.csv', index_col=[0,1],header=[0, 1]),
                        'no_work':pd.read_csv('../Data/base_contact_mtx/full_leisure_mtx_no_work.csv', index_col=[0,1],header=[0, 1])}

In [30]:
full_mtx_work['no_work']

11                                            \
age                  0-4         10-19         20-29         30-39   
area age                                                             
11   0-4    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     10-19  0.000000e+00  2.148416e+01  1.638141e+01  8.025447e+00   
     20-29  0.000000e+00  2.333217e+01  6.645899e+01  3.991187e+01   
     30-39  0.000000e+00  1.859981e+01  4.982823e+01  5.420356e+01   
     40-49  0.000000e+00  1.732018e+01  4.273015e+01  4.603202e+01   
...                  ...           ...           ...           ...   
71   40-49  0.000000e+00  5.576312e-04  1.375717e-03  1.482022e-03   
     5-9    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     50-59  0.000000e+00  5.112910e-04  9.439438e-04  1.074182e-03   
     60-69  8.207218e-09  4.267094e-05  1.165952e-04  1.227714e-04   
     70+    0.000000e+00  1.798339e-08  9.319916e-08  9.476366e-08   

                                                                    \
age                40-49           5-9         50-59         60-69   
area age                                                             
11   0-4    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     10-19  1.240165e+01  0.000000e+00  6.775155e+00  3.669036e-01   
     20-29  4.688450e+01  0.000000e+00  3.598898e+01  2.613599e+00   
     30-39  7.255148e+01  0.000000e+00  4.748975e+01  2.440348e+00   
     40-49  8.028032e+01  0.000000e+00  5.601615e+01  2.640837e+00   
...                  ...           ...           ...           ...   
71   40-49  2.584662e-03  0.000000e+00  1.803465e-03  8.502294e-05   
     5-9    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
     50-59  2.115049e-03  0.000000e+00  1.948521e-03  9.403783e-05   
     60-69  2.228248e-04  4.662224e-09  2.320017e-04  1.304597e-05   
     70+    6.349530e-08  0.000000e+00  5.559467e-08  4.020632e-08   

                         11_betshemesh  ...       62_arab        71            \
age                  70+           0-4  ...           70+       0-4     10-19   
area age                                ...                                     
11   0-4    2.275311e-03  0.000000e+00  ...  0.000000e+00  0.000000  0.000000   
     10-19  2.040730e-04  0.000000e+00  ...  2.636520e-08  0.000000  0.000179   
     20-29  7.801270e-04  0.000000e+00  ...  1.007885e-07  0.000000  0.000195   
     30-39  5.838037e-04  0.000000e+00  ...  7.542448e-08  0.000000  0.000155   
     40-49  1.048780e-03  0.000000e+00  ...  1.354971e-07  0.000000  0.000144   
...                  ...           ...  ...           ...       ...       ...   
71   40-49  3.376595e-08  0.000000e+00  ...  1.268142e-07  0.000000  3.757004   
     5-9    0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000  0.000000   
     50-59  2.944125e-08  0.000000e+00  ...  1.105720e-07  0.000000  3.444790   
     60-69  4.923508e-08  1.146204e-08  ...  1.849114e-07  0.000069  0.287493   
     70+    2.361953e-08  0.000000e+00  ...  8.870748e-08  0.000000  0.000121   

                                                                            \
age             20-29      30-39      40-49       5-9      50-59     60-69   
area age                                                                     
11   0-4     0.000000   0.000000   0.000000  0.000000   0.000000  0.000000   
     10-19   0.000235   0.000202   0.000236  0.000000   0.000105  0.000007   
     20-29   0.000955   0.001002   0.000891  0.000000   0.000559  0.000048   
     30-39   0.000716   0.001361   0.001379  0.000000   0.000737  0.000045   
     40-49   0.000614   0.001156   0.001526  0.000000   0.000870  0.000048   
...               ...        ...        ...       ...        ...       ...   
71   40-49  15.967184  30.069753  39.702632  0.000000  22.618534  1.258310   
     5-9     0.000000   0.000000   0.000000  0.000000   0.000000  0.001559   
     50-59  10.955836  21.794813  32.488974  0.000000  24.437776  1

In [31]:
C_calibration = {}

In [32]:
home_no_inter = []
work_no_inter = []
leis_no_inter = []

home_inter = []
work_inter = []
leis_inter = []

In [33]:
# no intervation are null groups
for i in range(1000):
    home_no_inter.append(np.zeros((180,180)))
    work_no_inter.append(np.zeros((180,180)))
    leis_no_inter.append(np.zeros((180,180)))

In [34]:
# Intervantion
# first days of routine from feb 21st
for i in range(9+14):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['routine'].values)
    leis_inter.append(full_mtx_leisure['routine'].values)    

# first days of no school from march 14th
for i in range(2):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['no_school'].values)
    leis_inter.append(full_mtx_leisure['no_school'].values)   

# without school and work
for i in range(1000-2-9-14):
    home_inter.append(full_mtx_home.values)
    work_inter.append(full_mtx_work['no_work'].values)
    leis_inter.append(full_mtx_leisure['no_work'].values) 

In [35]:
C_calibration['home_inter'] = home_no_inter
C_calibration['work_inter'] = work_no_inter
C_calibration['leisure_inter'] = leis_no_inter
C_calibration['home_non'] = home_inter
C_calibration['work_non'] = work_inter
C_calibration['leisure_non'] = leis_inter

In [36]:
# Save
with open('../Data/parameters/C_calibration.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Haredi vector

In [17]:
# Loading raw data
hared_dis = pd.read_csv('../Data/demograph/religion_dis.csv', index_col=0)[['cell_id','Orthodox']].copy()
hared_dis.set_index('cell_id', inplace=True)
hared_dis.head()

,Orthodox
cell_id,
11,0.010743
11_betshemesh,0.175258
21,0.020747
22,0.011432
23,0.033713


In [26]:
# Creating model orthodox dist. and save it as pickle
model_orthodox_dis = np.zeros(len(GA))
for i in GA.keys():
    model_orthodox_dis[i] = hared_dis.loc[str(GA[i][0])]
    
with open('../Data/parameters/orthodox_dist.pickle', 'wb') as handle:
    pickle.dump(model_orthodox_dis, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [27]:
model_orthodox_dis

array([0.01074322, 0.01074322, 0.01074322, 0.01074322, 0.01074322,
       0.01074322, 0.01074322, 0.01074322, 0.01074322, 0.17525794,
       0.17525794, 0.17525794, 0.17525794, 0.17525794, 0.17525794,
       0.17525794, 0.17525794, 0.17525794, 0.020747  , 0.020747  ,
       0.020747  , 0.020747  , 0.020747  , 0.020747  , 0.020747  ,
       0.020747  , 0.020747  , 0.01143183, 0.01143183, 0.01143183,
       0.01143183, 0.01143183, 0.01143183, 0.01143183, 0.01143183,
       0.01143183, 0.03371292, 0.03371292, 0.03371292, 0.03371292,
       0.03371292, 0.03371292, 0.03371292, 0.03371292, 0.03371292,
       0.02046316, 0.02046316, 0.02046316, 0.02046316, 0.02046316,
       0.02046316, 0.02046316, 0.02046316, 0.02046316, 0.06416474,
       0.06416474, 0.06416474, 0.06416474, 0.06416474, 0.06416474,
       0.06416474, 0.06416474, 0.06416474, 0.00151861, 0.00151861,
       0.00151861, 0.00151861, 0.00151861, 0.00151861, 0.00151861,
       0.00151861, 0.00151861, 0.16962388, 0.16962388, 0.16962